# Heat Equation Solver using most recent API

## What's new?


- using the `XDGTimestepper` to perform a high-order, implicit time integration on the Heat equation
- a comparison of the error computed against the exact solution vs. the error computed against the numerical solution with the finest timestep

## Prerequisites

- implementation of the Symmetrical Interior Penalty (SIP) form for the Laplace operator,  chapter **SIP**


## Problem statement

We are investigating the instationary Heat equation
$$
  \partial_t u - \Delta u = 0 \text{ in } (x,y) \in (-1,1)^2, \ \ \ u = 0 \text{ on }  \partial (-1,1)^2
$$
with the initial value
$$
    u_0(x,y) = \cos(x \pi / 2) \cos(y \pi / 2),
$$
and the exact solution
$$
    u_{\text{ex}}(t,x,y) = ( -\pi^2 / 2 ) u_0(x,y).
$$

## Solution within the BoSSS framework

First, we initialize the new worksheet;
Note: 
1. This tutorial can be found in the source code repository as as `HeatEquationSolver.ipynb`. 
   One can directly load this into Jupyter to interactively work with the following code examples.
2. **In the following line, the reference to `BoSSSpad.dll` is required**. 
   You must either set `#r "BoSSSpad.dll"` to something which is appropirate for your computer
   (e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net6.0\BoSSSpad.dll` if you installed the binary distribution),
   or, if you are working with the source code, you must compile `BoSSSpad` and put it side-by-side to this worksheet file
   (from the original location in the repository, you can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
using BoSSS.Solution.XdgTimestepping;

## Creation of Grid and Differential Operator

Here, 1 2D, $16 \times 16$ mesh is used:

In [3]:
var grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(-1,1,17), GenericBlas.Linspace(-1,1,17));

Regarding the spatial part of the operator, we re-use the SIP-implementation 
which is already available in the `BoSSS.Solution.NSECommon` library.
For this implementation, whe have to specify the diffusion coefficient (here to -1.0)
as well as the Location of the Dirichlet boundary.

In [1]:
class Laplace : BoSSS.Solution.NSECommon.SIPLaplace {

    public Laplace() : base(1.4, "u") { } // override the constuctor; 
    //                                       The factor 1.4 is a constant "safty factor" multiper for the penalty parameter.
    //                                       The base implementation takes care about all other penalty factor dependencies, 
    //                                       i.e. dependence of the penatly factor on local cell size, DG polynomial degree and cell shape.
    //                                       The second argument, "u", is the name of the variable for the `ArgumentOrdering`.


    // Specifies whetherss a specific point (`inp.X`) is either a Dirichlet or a Neumann boundary;
    // Since in our case ther, the entire boundary should be Diriclet, we always return true.
    protected override bool IsDirichlet(ref CommonParamsBnd inp) {
        return true;
    }
    
    // Diffusion coefficient
    override public double Nu(double[] x, double[] p, int jCell) {
        return -1.0;
    }

}

Error: (1,17): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)
(12,45): error CS0246: The type or namespace name 'CommonParamsBnd' could not be found (are you missing a using directive or an assembly reference?)
(3,24): error CS1729: 'BoSSS.Solution.NSECommon.SIPLaplace' does not contain a constructor that takes 2 arguments

In [5]:
var Op = new DifferentialOperator(1,0,1,QuadOrderFunc.Linear(), "u", "R1");
Op.EquationComponents["R1"].Add(new Laplace());

Op.TemporalOperator = new ConstantTemporalOperator(Op);
Op.IsLinear = true;
Op.Commit();

In [6]:
SinglePhaseField TimeIntegrate(SinglePhaseField u0, double EndTime, int NoOfTimeteps) {
    var u1 = u0.CloneAs();
    var Timestepper = new XdgTimestepping(Op, new DGField[] { u1 }, new DGField[] { new SinglePhaseField(u0.Basis)}, 
                            TimeSteppingScheme.SDIRK_54);


    double dt = EndTime/NoOfTimeteps;

    for(int i = 0; i < NoOfTimeteps; i++)
        Timestepper.Solve(dt*i, dt);
    
    return u1;
}

In [7]:
var u0 = new SinglePhaseField(new Basis(grid, 4), "u");
u0.ProjectField((double[] X) => Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5));
//u0 = TimeIntegrate(u0, 1, 10); // regularize, somehow

In [8]:
Tecplot("u0", u0);

Supersampling 4 requested, but limiting to 3 because higher values would very likely exceed this computers memory.
Note: Higher supersampling values are supported by external plot application, or by using e.g. the Tecplot class directly.
Writing output file C:\Users\flori\Desktop\Timestepping\u0...
done.


In [ ]:
//for(int i = 0; i < 10; i++) {
//    Console.WriteLine("   --------------  timestep: " + (i + 1));
//    var u1 = TimeIntegrate(u0, 0.1, 1);
//    Tecplot("u" + (i+1), u1);
//    u0 = u1;
//}

In [10]:
var NoOfTimestepS = new int[] { 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000/*, 2000, 5000*/ };
var solutions = new List<SinglePhaseField>();
for(int i = 0; i < NoOfTimestepS.Length; i++) {
    Console.WriteLine("   --------------  computing no of timeteps: " + NoOfTimestepS[i]);
    var u1 = TimeIntegrate(u0, 0.1, NoOfTimestepS[i]);
    //Tecplot("u1." + i, u1);
    solutions.Add(u1);
}

   --------------  computing no of timeteps: 1
   --------------  computing no of timeteps: 2
   --------------  computing no of timeteps: 5
   --------------  computing no of timeteps: 10
   --------------  computing no of timeteps: 20
   --------------  computing no of timeteps: 50
   --------------  computing no of timeteps: 100
   --------------  computing no of timeteps: 200
   --------------  computing no of timeteps: 500
   --------------  computing no of timeteps: 1000


In [ ]:
for(int i = 0; i < solutions.Count; i++)
    Tecplot("u1." + i, solutions[i]);

In [ ]:
double uExEq(double[] X) {
   double Lambda = -Math.PI.Pow2()*0.5;
   double tend = 0.1;
   double u0 = Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5);
   return Math.Exp(Lambda*tend)*u0;
}

In [13]:
var uEx = solutions.Last();

In [ ]:
var Errs = solutions.Take(solutions.Count - 1).Select(u => u.L2Error(uEx)).ToArray();

In [ ]:
Errs

In [ ]:
double[] diffs = new double[Errs.Length];
for(int i = 0; i < diffs.Length; i++) {
    diffs[i] = solutions[i].L2Error(solutions[i + 1]);
}

In [ ]:
diffs

In [ ]:
double[] ErrEx = new double[Errs.Length];
for(int i = 0; i < diffs.Length; i++) {
    ErrEx[i] = solutions[i].L2Error(uExEq);
}

In [ ]:
ErrEx

In [ ]:
double ErrHi(SinglePhaseField u) {
    SinglePhaseField uLo = new SinglePhaseField(new Basis(u.Basis.GridDat, u.Basis.Degree - 1));
    uLo.AccLaidBack(1.0, u);
    var uHi = u.CloneAs();
    uHi.AccLaidBack(-1.0, uLo);
    return uHi.L2Norm();
}

In [ ]:
double[] HiDeg = new double[Errs.Length];
for(int i = 0; i < HiDeg.Length; i++) {
    HiDeg[i] = ErrHi(solutions[i]);
}

In [ ]:
HiDeg

In [ ]:
var gp = new Gnuplot();
double[] ts = NoOfTimestepS.Take(NoOfTimestepS.Length - 1).Select(x => (double)x).ToArray();
gp.PlotLogXLogY(ts, Errs, title:"Exp Error", format:new PlotFormat("-xk"));
gp.PlotLogXLogY(ts, diffs, title:"Diffs", format:new PlotFormat("-ob"));
gp.PlotLogXLogY(ts, ErrEx, title:"Exact Error", format:new PlotFormat("-+r"));
gp.PlotLogXLogY(ts, HiDeg, title:"High Degree Norm", format:new PlotFormat(":ok"));
gp.PlotNow()

In [ ]:
ts.LogLogRegressionSlope(Errs)